In [6]:
import pickle
import pandas as pd
import tabulate

In [2]:
def pickle_load(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

    
def load(t3f_cpu_path, t3f_gpu_path, ttpy_path=None):
    cpu = pickle_load(t3f_cpu_path)
    gpu = pickle_load(t3f_gpu_path)
    d = {}
    op_list = ['matvec', 'matmul', 'norm', 'round', 'gram', 'project_rank100']
    
    one_cpu = []
    for op in op_list:
        if op == 'gram':
            # Single element Gram matrix is flat inner.
            curr_time = cpu['flatinner']['wall_time'] * 1000
        else:
            curr_time = cpu[op]['wall_time'] * 1000
        one_cpu.append(curr_time)
    d['one on CPU'] = one_cpu
    
    one_gpu = []
    for op in op_list:
        if op == 'gram':
            # Single element Gram matrix is flat inner.
            curr_time = gpu['flatinner']['wall_time'] * 1000
        else:
            curr_time = gpu[op]['wall_time'] * 1000
        one_gpu.append(curr_time)
    d['one on GPU'] = one_gpu
    
    batch_cpu = []
    for op in op_list:
        if op == 'gram':
            curr_time = cpu['batch_gram']['wall_time'] * 1000 / 100**2
        else:
            curr_time = cpu['batch_' + op]['wall_time'] * 1000 / 100
        batch_cpu.append(curr_time)
    d['batch on CPU'] = batch_cpu
    
    batch_gpu = []
    for op in op_list:
        if op == 'gram':
            curr_time = gpu['batch_gram']['wall_time'] * 1000 / 100.**2
        else:
            curr_time = gpu['batch_' + op]['wall_time'] * 1000 / 100.
#         curr_time = gpu['batch_' + op + '_time'] * 1000
        batch_gpu.append(curr_time)
    d['batch on GPU'] = batch_gpu
    
    if ttpy_path:
        ttpy = pickle_load(ttpy_path)
        ttpy_col = []
        for op in op_list:
            if op == 'gram':
                # Single element Gram matrix is flat inner.
                curr_time = ttpy['flatinner_time'] * 1000
            else:
                curr_time = ttpy[op + '_time'] * 1000
            ttpy_col.append(curr_time)
    d['ttpy, one on CPU'] = ttpy_col
    
    df = pd.DataFrame(d, index=op_list)
    regime_list = ['one on CPU', 'one on GPU', 'batch on CPU', 'batch on GPU']
    if ttpy_path:
        regime_list = ['ttpy, one on CPU'] + regime_list
    df = df[regime_list]
    return df

In [3]:
df = load('logs_cpu.pkl', 'logs_gpu.pkl', ttpy_path='logs_ttpy.pkl')

In [4]:
df.round(decimals=3)

,"ttpy, one on CPU",one on CPU,one on GPU,batch on CPU,batch on GPU
matvec,11.142,1.190,0.744,1.885,0.140
matmul,86.191,9.849,0.950,17.483,1.461
norm,3.790,2.136,1.019,0.253,0.044
round,73.027,86.040,165.969,8.234,161.102
gram,0.145,0.606,0.973,0.021,0.001
project_rank100,116.868,3.001,13.239,1.645,0.226


In [5]:
print(df.round(decimals=3).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &  ttpy, one on CPU &  one on CPU &  one on GPU &  batch on CPU &  batch on GPU \\
\midrule
matvec          &            11.142 &       1.190 &       0.744 &         1.885 &         0.140 \\
matmul          &            86.191 &       9.849 &       0.950 &        17.483 &         1.461 \\
norm            &             3.790 &       2.136 &       1.019 &         0.253 &         0.044 \\
round           &            73.027 &      86.040 &     165.969 &         8.234 &       161.102 \\
gram            &             0.145 &       0.606 &       0.973 &         0.021 &         0.001 \\
project\_rank100 &           116.868 &       3.001 &      13.239 &         1.645 &         0.226 \\
\bottomrule
\end{tabular}



In [10]:
print(tabulate.tabulate(df.round(decimals=3), headers=df.keys(), tablefmt="rst"))

===============  ==================  ============  ============  ==============  ==============
..                 ttpy, one on CPU    one on CPU    one on GPU    batch on CPU    batch on GPU
===============  ==================  ============  ============  ==============  ==============
matvec                       11.142         1.19          0.744           1.885           0.14
matmul                       86.191         9.849         0.95           17.483           1.461
norm                          3.79          2.136         1.019           0.253           0.044
round                        73.027        86.04        165.969           8.234         161.102
gram                          0.145         0.606         0.973           0.021           0.001
project_rank100             116.868         3.001        13.239           1.645           0.226
===============  ==================  ============  ============  ==============  ==============
